In [8]:
import os

import pandas as pd
import gspread_dataframe as gs

# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

import sistemas
import sei
import functions
import page
from page import *

% reload_ext autoreload
% autoreload 2

In [ ]:
gc = functions.authenticate_google("files/anatel.json")
wb = gc.open(title='RC')

In [61]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [62]:
browser = sei.login_sei(browser, usr, pwd)

In [64]:
sei_ = sei.Sei(browser, sei_.get_processos())

sei_.itera_processos()

In [59]:
numero = "53504.005634/2018-81"

In [ ]:
sht = wb.worksheet(numero)

In [ ]:
df = gs.get_as_dataframe(sht, dtype=str)

In [ ]:
p = sei_.go_to_processo(numero)

In [41]:
tipo = "RC_Oficio de Cassação"

In [42]:
p.incluir_oficio(tipo)

In [21]:
from bs4 import BeautifulSoup as soup

html = soup("""          
<option value="null"> </option><option value="40">Controle Interno (Art. 26, § 3º, da Lei nº 10.180/2001)</option><option value="45">Direito Autoral (Art. 24, III, da Lei nº 9.610/1998)</option><option value="33">Documento Preparatório (Art. 7º, § 3º, da Lei nº 12.527/2011)</option><option value="35">Fiscalização / Investigação da Anatel (Art. 174 da Lei nº 9.472/1997)</option><option value="34">Informação Pessoal (Art. 31 da Lei nº 12.527/2011)</option><option value="39">Informações Contábeis de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997)</option><option value="38">Informações Econômico-Financeiras de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997)</option><option value="37">Informações Operacionais de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997)</option><option value="46">Informações Privilegiadas de Sociedades Anônimas (Art. 155, § 2º, da Lei nº 6.404/1976)</option><option value="36">Informações Técnicas de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997)</option><option value="41">Interceptação de Comunicações Telefônicas (Art. 8º, caput, da Lei nº 9.296/1996)</option><option value="31">Investigação de Responsabilidade de Servidor (Art. 150 da Lei nº 8.112/1990)</option><option value="47">Livros e Registros Contábeis Empresariais (Art. 1.190 do Código Civil)</option><option value="48">Operações Bancárias (Art. 1º da Lei Complementar nº 105/2001)</option><option value="49">Proteção da Propriedade Intelectual de Software (Art. 2º da Lei nº 9.609/1998)</option><option value="43">Protocolo -Pendente Análise de Restrição de Acesso (Art. 6º, III, da Lei nº 12.527/2011)</option><option value="50">Segredo de Justiça no Processo Civil (Art. 189 do Código de Processo Civil)</option><option value="51">Segredo de Justiça no Processo Penal (Art. 201, § 6º, do Código de Processo Penal)</option><option value="52">Segredo Industrial (Art. 195, XIV, Lei nº 9.279/1996)</option><option value="53">Sigilo das Comunicações (Art. 3º, V, da Lei nº 9.472/1997)</option><option value="54">Sigilo de Empresa em Situação Falimentar (Art. 169 da Lei nº 11.101/2005)</option><option value="55">Sigilo do Inquérito Policial (Art. 20 do Código de Processo Penal)</option><option value="42">Situação Econômico-Financeira de Sujeito Passivo (Art. 198, caput, da Lei nº 5.172/1966 - CTN)</option>
        """, "lxml")

In [22]:
tipos = [t.string for t in html.descendants if t.string not in (None, '\xa0', '\n')]
tipos

[' ',
 ' ',
 'Controle Interno (Art. 26, § 3º, da Lei nº 10.180/2001)',
 'Controle Interno (Art. 26, § 3º, da Lei nº 10.180/2001)',
 'Direito Autoral (Art. 24, III, da Lei nº 9.610/1998)',
 'Direito Autoral (Art. 24, III, da Lei nº 9.610/1998)',
 'Documento Preparatório (Art. 7º, § 3º, da Lei nº 12.527/2011)',
 'Documento Preparatório (Art. 7º, § 3º, da Lei nº 12.527/2011)',
 'Fiscalização / Investigação da Anatel (Art. 174 da Lei nº 9.472/1997)',
 'Fiscalização / Investigação da Anatel (Art. 174 da Lei nº 9.472/1997)',
 'Informação Pessoal (Art. 31 da Lei nº 12.527/2011)',
 'Informação Pessoal (Art. 31 da Lei nº 12.527/2011)',
 'Informações Contábeis de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997)',
 'Informações Contábeis de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997)',
 'Informações Econômico-Financeiras de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997)',
 'Informações Econômico-Financeiras de Empresa (Art. 39, parágrafo único, da Lei nº 9.472/1997

SyntaxError: can't use starred expression here (<ipython-input-45-a16d64335f12>, line 1)